In [1]:
!pip install -q Pillow

In [2]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf

## Data Gathering

Download dataset and extract to `train_data` and `val_data`.

In [3]:
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
!wget http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip

--2020-12-27 04:32:28--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 74.125.199.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘horse-or-human.zip’

horse-or-human.zip  100%[===================>] 142.65M   176MB/s    in 0.8s    

2020-12-27 04:32:29 (176 MB/s) - ‘horse-or-human.zip’ saved [149574867/149574867]

--2020-12-27 04:32:29--  http://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [appl

In [4]:
TRAIN_DIR = "./train_data"
VAL_DIR = "./val_data"
!unzip -q horse-or-human.zip -d $TRAIN_DIR
!unzip -q validation-horse-or-human.zip -d $VAL_DIR

Read a sample image and check pixel range.

In [5]:
train_sample_fn = os.listdir(os.path.join(TRAIN_DIR, "horses"))[0]
train_sample_path = os.path.join(TRAIN_DIR, "horses", train_sample_fn)

**Note:** The last channel is transparency.

In [6]:
img = Image.open(train_sample_path)
img_data = np.asarray(img)

print("Image shape:", img_data.shape)
print("Pixel in range:", np.min(img_data), np.max(img_data))

print(img_data[:1])

Image shape: (300, 300, 4)
Pixel in range: 1 255
[[[158 178 196 255]
  [159 178 196 255]
  [159 178 196 255]
  ...
  [137 139 147 255]
  [137 138 147 255]
  [137 138 146 255]]]


## Data Flow

In [7]:
img_size = (300, 300)
img_shape = (*img_size, 3)   

Create data flow from a directory, each image is scaled to the range 0-255.

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def gen_new_data(data_folder):
  data_gen = ImageDataGenerator(rescale=1./255)
  data = data_gen.flow_from_directory(
      data_folder,
      target_size=img_size,
      batch_size=64,
      class_mode="binary",
  )
  return data

In [9]:
train_data_gen = gen_new_data(TRAIN_DIR)
val_data_gen = gen_new_data(VAL_DIR)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


## Load Pretrained Model

Load InceptionV3 pretrained on ImageNet and freeze all layers above `mixed7`. This layer will give us features extracted from InceptionV3. We're gonna replace the top of the model with our own classifier.

In [10]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
pretrained_model = InceptionV3(include_top=False, weights="imagenet", input_shape=(300, 300, 3))

87916544/87910968 [==============================] - 1s 0us/step


In [11]:
# Freeze all layers
for layer in pretrained_model.layers:
  layer.trainable = False

# We keep all layers up to `mixed7`
last_pretrained_layer = pretrained_model.get_layer(name="mixed7")
print("Last layer shape:", last_pretrained_layer.output_shape)

Last layer shape: (None, 17, 17, 768)


## Model Architecture & Training

In [12]:
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [13]:
train_model = Flatten()(last_pretrained_layer.output)
train_model = Dense(1024, activation="swish")(train_model)
train_model = Dropout(0.1)(train_model)
train_model = Dense(1, activation="sigmoid")(train_model)

In [14]:
model = Model(pretrained_model.input, train_model)

In [15]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=1e-4)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["acc"])
model.fit(train_data_gen, epochs=5, validation_data=val_data_gen)

Epoch 1/5
17/17 [==============================] - 20s 591ms/step - loss: 28.0895 - acc: 0.8036 - val_loss: 0.4264 - val_acc: 0.9492
Epoch 2/5
17/17 [==============================] - 9s 515ms/step - loss: 0.0011 - acc: 0.9996 - val_loss: 0.1311 - val_acc: 0.9805
Epoch 3/5
17/17 [==============================] - 9s 508ms/step - loss: 2.0649e-04 - acc: 1.0000 - val_loss: 0.1690 - val_acc: 0.9844
Epoch 4/5
17/17 [==============================] - 9s 516ms/step - loss: 1.3674e-05 - acc: 1.0000 - val_loss: 0.1651 - val_acc: 0.9844
Epoch 5/5
17/17 [==============================] - 9s 514ms/step - loss: 6.3944e-06 - acc: 1.0000 - val_loss: 0.1604 - val_acc: 0.9883


In [16]:
model.evaluate(val_data_gen)

4/4 [==============================] - 1s 262ms/step - loss: 0.1604 - acc: 0.9883


[0.1604381650686264, 0.98828125]

In [18]:
model.save("mymodel.h5")